In [1]:
#imports
from pybaseball import statcast, schedule_and_record
from pybaseball import statcast_single_game
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# collect Statcast data on all pitches from the months of May and June
data = schedule_and_record(2018, 'TBR')
print(data)

                  Date   Tm Home_Away  Opp   W/L     R    RA   Inn    W-L  \
1     Thursday, Mar 29  TBR      Home  BOS     W   6.0   4.0   9.0    1-0   
2       Friday, Mar 30  TBR      Home  BOS     L   0.0   1.0   9.0    1-1   
3     Saturday, Mar 31  TBR      Home  BOS     L   2.0   3.0   9.0    1-2   
4        Sunday, Apr 1  TBR      Home  BOS     L   1.0   2.0   9.0    1-3   
5       Tuesday, Apr 3  TBR         @  NYY     L   4.0  11.0   9.0    1-4   
6     Wednesday, Apr 4  TBR         @  NYY     L   2.0   7.0   9.0    1-5   
7      Thursday, Apr 5  TBR         @  BOS  L-wo   2.0   3.0  12.0    1-6   
8      Saturday, Apr 7  TBR         @  BOS     L   3.0  10.0   9.0    1-7   
9        Sunday, Apr 8  TBR         @  BOS     L   7.0   8.0   9.0    1-8   
10       Monday, Apr 9  TBR         @  CHW     W   5.0   4.0   9.0    2-8   
11     Tuesday, Apr 10  TBR         @  CHW     W   6.0   5.0   9.0    3-8   
12   Wednesday, Apr 11  TBR         @  CHW     L   1.0   2.0   9.0    3-9   

In [3]:
data = statcast_single_game(529406)

ValueError: could not convert string to float: 'Standard'

In [4]:
# data1 = statcast('2018-04-01', '2018-04-10' )

data1 = statcast('2018-04-01', '2018-04-10')

This is a large query, it may take a moment to complete
Completed sub-query from 2018-04-01 to 2018-04-06
Completed sub-query from 2018-04-07 to 2018-04-10


ValueError: could not convert string to float: 'Standard'

In [3]:
# drop rows where the data we need is missing
pd.options.mode.chained_assignment = None
data2 = data[pd.notnull(data['launch_angle'])]
data2 = data2[pd.notnull(data['launch_speed'])]
data2 = data2[pd.notnull(data['estimated_ba_using_speedangle'])]
data2 = data2[pd.notnull(data['estimated_woba_using_speedangle'])]
data2 = data2[pd.notnull(data['pitch_type'])]
list(data)

NameError: name 'data' is not defined

In [ ]:
plt.scatter(data2['launch_speed'], data2['launch_angle'], c=data2['estimated_ba_using_speedangle'])
plt.xlabel('Exit Velocity', fontsize=14)
plt.ylabel('Launch Angle', fontsize=14)
plt.colorbar().set_label('Hit Probability', rotation=270)
plt.xlim(20, 140)
plt.ylim(-25, 100)
plt.title("Hit probability by Launch Angle and Exit Velocity");

In [ ]:
plt.scatter(data2['launch_speed'], data2['launch_angle'], c=data2['estimated_woba_using_speedangle'])
plt.xlabel('Exit Velocity', fontsize=14)
plt.ylabel('Launch Angle', fontsize=14)
plt.colorbar().set_label('wOBA Estimate', rotation=270)
plt.xlim(20, 140)
plt.ylim(-25, 100)
plt.title("wOBA Estimate by Launch Angle and Exit Velocity");

In [ ]:
# now just the home runs 
data2['hr'] = 0
data2.loc[data2['events'] == 'home_run', 'hr'] = 1
data3 = data2.loc[data2['hr'] ==1]
data3.shape

In [ ]:
plt.scatter(data3['launch_speed'], data3['launch_angle'],c='red')
plt.xlabel('Exit Velocity', fontsize=14)
plt.ylabel('Launch Angle', fontsize=14)
plt.xlim(20, 140)
plt.ylim(-25, 100)
plt.title("Home Runs by Launch Angle and Exit Velocity");

In [ ]:
#create launch speed bins
bins = np.linspace(data2.launch_speed.min(), data2.launch_speed.max(), 5)
groups = data2.groupby(np.digitize(data2.launch_speed, bins))

# take alook at how various features vary with launch speed
print(groups)

In [ ]:
# re-bin with a value more useful for plotting
bins = np.linspace(data2.launch_speed.min(), data2.launch_speed.max(), 30)
groups = data2.groupby(np.digitize(data2.launch_speed, bins))
bin_woba = groups['estimated_woba_using_speedangle'].mean()

In [ ]:
plt.plot(bins,bin_woba)
plt.xlabel('Launch Angle', fontsize=14)
plt.ylabel('Expected wOBA Value', fontsize=14)
plt.xlim(40,120)